In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sbs

In [4]:
otrs = pd.read_excel("C:\curso_python\pandas\otrs_csv.xlsx")

In [5]:
otrs

,Ticket#,Title,Created,Queue,State,Customer User,Customer ID,Type,Accounted time,StateType,Impact,DownTime ( Min ),Alcance,Pedido - HDR
0,2023050678000078,ERROR AL COBRAR PEDIDO /CAS-3064617,2023-05-06 11:14:31 (America/Argentina/Buenos_...,Sitio::Problemas con el Cobro::Cobrado y devuelto,open,cquintana,Sucursal 090,Incident,10,open,NaN,NaN,NaN,NaN
1,2023050878000252,Denuncia. COPREC. EX-2023- 46397670. SILVA FER...,2023-05-08 13:50:03 (America/Argentina/Buenos_...,Sitio::Problemas con el Cobro::Cobrado y devuelto,new,LJBARONE,AgustinEzequielGomez@coto.com.ar,Incident,0,new,NaN,NaN,NaN,NaN
2,2023050878000314,Monitores de sectores / suc 184 (en tema Defil...,2023-05-08 17:20:12 (America/Argentina/Buenos_...,BackOffice::Monitores,new,CBCHURQUINA,Sucursal 184,Incident,15,new,NaN,NaN,NaN,NaN
3,2023051078000221,Primera Compra mayores a 3 meses no funciona (...,2023-05-10 15:20:19 (America/Argentina/Buenos_...,Sitio,new,172778,Casa Central,Incident,10,new,NaN,NaN,NaN,NaN
4,2023051678000406,no figura dni tarjeta en pedido 21051109 (en t...,2023-05-16 17:53:53 (America/Argentina/Buenos_...,BackOffice::Mesa de Control::No figuran datos ...,open,CarKMori,Sucursal 075,Incident,0,open,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,2023061478000131,21265206 / suc 197 / 08-06-2023 00:05 / descue...,2023-06-14 14:26:13 (America/Argentina/Buenos_...,Multistore::Promociones::No aplican promocione...,open,lrdiaz2,Sucursal 197,Incident,2,open,NaN,0.0,Pedido-HDR,21265206.0
753,2023061478000141,21207671 / suc 185 /,2023-06-14 14:28:00 (America/Argentina/Buenos_...,Multistore::Negociados::Causa: Promocion no ap...,closed successful,LGQUIROGA,Sucursal 185,Incident,17,closed,NaN,0.0,Pedido-HDR,21207671.0
754,2023061478000159,21315138 / suc 085 /,2023-06-14 14:29:15 (America/Argentina/Buenos_...,BackOffice::Operaciones de Sucursal::Errores O...,closed successful,LGQUIROGA,Sucursal 185,Incident,12,closed,NaN,0.0,Pedido-HDR,21315138.0
755,2023061478000168,21207701 / suc 185 /,2023-06-14 14:30:20 (America/Argentina/Buenos_...,Multistore::Negociados::Causa: Promocion no ap...,closed successful,LGQUIROGA,Sucursal 185,Incident,17,closed,NaN,0.0,Pedido-HDR,21207701.0


In [7]:
otrs = otrs.rename(
    columns = {
        "DownTime ( Min )" : "Downtime"
    }
)

In [8]:
#creando downtime de track
track_downtime = otrs[otrs["Queue"].str.contains("Track3")].groupby("Queue",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending = False).loc[lambda x : x["Downtime"] > 0]

In [9]:
track_downtime

,Queue,Downtime
5,Track3::Contenedor PreRuteo,203.0
9,Track3::Interfaces::Track3-> Envios,162.0
0,Track3,42.0
3,Track3::APk EMC::No puedo finalizar una HDR,23.0
8,Track3::FrontEnd Track3 - Digital,5.0


In [14]:
#creando downtime de backoffice
backoffice_downtime = otrs[otrs["Queue"].str.contains("BackOffice")].groupby("Queue",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x: x["Downtime"] > 0]

In [12]:
backoffice_downtime

,Queue,Downtime
5,Track3::Contenedor PreRuteo,203.0
9,Track3::Interfaces::Track3-> Envios,162.0
0,Track3,42.0
3,Track3::APk EMC::No puedo finalizar una HDR,23.0
8,Track3::FrontEnd Track3 - Digital,5.0


In [16]:
#creando downtime de operaciones
ncd_downtime = otrs[otrs["Queue"].str.contains("Operaciones Coto Digital")].groupby("Queue",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x:x["Downtime"]>0]

In [19]:
track_downtime

,Queue,Downtime
5,Track3::Contenedor PreRuteo,203.0
9,Track3::Interfaces::Track3-> Envios,162.0
0,Track3,42.0
3,Track3::APk EMC::No puedo finalizar una HDR,23.0
8,Track3::FrontEnd Track3 - Digital,5.0


In [21]:
ncd_downtime

,Queue,Downtime
10,Operaciones Coto Digital::Operaciones::Edicion...,265.0
11,Operaciones Coto Digital::Operaciones::Error O...,235.0
0,Operaciones Coto Digital,105.0
1,Operaciones Coto Digital::Aplicacion::CheckOut,85.0
9,Operaciones Coto Digital::Datos::Item Faltantes,70.0
5,Operaciones Coto Digital::Aplicacion::Picking:...,65.0
4,Operaciones Coto Digital::Aplicacion::Picking,30.0
8,Operaciones Coto Digital::Datos::Datos Pedidos,25.0
3,Operaciones Coto Digital::Aplicacion::Login,20.0
6,Operaciones Coto Digital::Aplicacion::Picking:...,11.0


In [22]:
backoffice_downtime

,Queue,Downtime
0,BackOffice,182.0
69,BackOffice::Simap,90.0
28,BackOffice::Mesa de Ticket::Error al realizar ...,88.0
52,BackOffice::Procesos::Job::Falla Job CDIGICUPO...,60.0
31,BackOffice::Operaciones de Sucursal::Errores O...,20.0
17,"BackOffice::Mesa de Control::Error ""ticket no ...",3.0
